In [2]:
!pip install https://artifactory.paypalinc.com/artifactory/api/pypi/python/packages/packages/04/57/f1cf3cf20fa78a3dcd6c7cc95e9f635bdf4fe12f00894d648b0cb5e42e54/trl-0.18.2-py3-none-any.whl

Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
  Using cached https://artifactory.paypalinc.com/artifactory/api/pypi/python/packages/packages/04/57/f1cf3cf20fa78a3dcd6c7cc95e9f635bdf4fe12f00894d648b0cb5e42e54/trl-0.18.2-py3-none-any.whl (366 kB)


In [3]:
!pip install peft --user

Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple


In [4]:
!pip install -U bitsandbytes --user

Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
  Using cached https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/packages/packages/db/9d/9382259196d7ad7f3550702390081224e673a705e75b5660ee377b592fc0/bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)


In [3]:
!pip install tf_keras --user

Looking in indexes: https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/simple
  Using cached https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/packages/packages/8a/ed/e08afca471299b04a34cd548e64e89d0153eda0e6cf9b715356777e24774/tf_keras-2.18.0-py3-none-any.whl (1.7 MB)
  Using cached https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/packages/packages/84/76/c55967ac9968ddaede25a4dce37aba37e9030656f02c12676151ce1b6f22/tensorflow-2.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.4 MB)
  Using cached https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/packages/packages/a8/45/2ebbde52ad2be18d3675b6bee50e68cd73c9e0654de77d595540b5129df8/protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Using cached https://dtr-vip.ccg13.slc.paypalinc.com/artifactory/api/pypi/python/packages/packages/b1/de/021c1d407befb505791764ad2cbd56ceaaa53a746baed01d2e2143f05f18/tensorboard-2.18.0-py3-none-any.whl (5.5 MB

In [1]:
!nvidia-smi

Mon Jun 23 17:24:45 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          Off | 00000000:8C:00.0 Off |                    0 |
| N/A   31C    P0              66W / 700W |      0MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import torch

In [4]:
# model_path = '/projects/llm-repo/models/Qwen/Qwen2.5-7B-Instruct'
model_path = '/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto"
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 8/8 [00:26<00:00,  3.27s/it]


In [7]:
model = prepare_model_for_kbit_training(model)

In [8]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model,lora_config)

In [9]:
# dataset = load_dataset("json", data_files="/projects/asankaranarayana/Fine_Tune_Hackathon/v1/task_trajectories.jsonl", split="train")
dataset = load_dataset("json", data_files="/projects/asankaranarayana/Fine_Tune_Hackathon/v1/task_trajectories_v2.jsonl", split="train")


Generating train split: 70000 examples [00:00, 212315.00 examples/s]


In [10]:
split_dataset = dataset.train_test_split(test_size=0.15, seed=42)

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [11]:
model.print_trainable_parameters()

trainable params: 68,812,800 || all params: 14,838,846,464 || trainable%: 0.4637


In [12]:
training_args = TrainingArguments(
    # output_dir="./qwen_venmo_sft_v1",
    output_dir="./qwen_venmo_sft_v2",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    lr_scheduler_type='cosine',
    report_to="none",
)

In [13]:
trainer = SFTTrainer(
    model = model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=lora_config,
    processing_class=tokenizer,
    args=training_args
)

Truncating eval dataset: 100%|██████████| 10500/10500 [00:00<00:00, 44460.52 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/envs/py311/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.794500
20,1.523200


In [ ]:
# trainer.model.save_pretrained('./qwen_finetuned_v1')
# tokenizer.save_pretrained('./qwen_finetuned_v1')
trainer.model.save_pretrained('./qwen_finetuned_v2')
tokenizer.save_pretrained('./qwen_finetuned_v2')

In [ ]:
from peft import PeftModel

In [ ]:
# base_model_path = "/projects/llm-repo/models/Qwen/Qwen2.5-7B-Instruct"  # or your local base path
# adapter_path = "./qwen_finetuned_v1"  # your trained LoRA output dir

base_model_path = '/projects/llm-repo/models/Qwen/Qwen2.5-14B-Instruct'
adapter_path = "./qwen_finetuned_v2"

# Load base model and tokenizer
model = AutoModelForCausalLM.from_pretrained(base_model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Load LoRA adapter on top of base
model = PeftModel.from_pretrained(model, adapter_path)

# Merge the LoRA weights into the base model
model = model.merge_and_unload()


In [ ]:
# output_path = "./qwen-merged"
output_path = "./qwen-merged-v2"

model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)